In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import spacy

In [ ]:
# Importing the datasets
df_train= pd.read_csv("https://raw.githubusercontent.com/alexerne-git/ML-Zoom/master/Data/training_data.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/alexerne-git/ML-Zoom/master/Data/test_data.csv")



In [ ]:
df_train.head(10)

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train

In [ ]:
df_test

# An aventure in the data

We want to understand a little more about our data.

Distribution, similarity between the test set and the train set, missing values and other stuff.




## Location

In [ ]:
# Plot NaN values to see the distribution

NAn= len(df_train.location[df_train["location"].isnull()==True])
Val= len(df_train.location)-NAn
#Piechart
labels = 'Nan', 'Values'
sizes = [NAn, Val]
colors = ['gold', 'lightcoral']
explode = (0.1, 0)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()
NAn

In [ ]:
NAn= len(df_test.location[df_test["location"].isnull()==True])
Val= len(df_test.location)-NAn
#Piechart
labels = 'Nan', 'Values'
sizes = [NAn, Val]
colors = ['gold', 'lightcoral']
explode = (0.1, 0)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()


A lot of Missing values. Maybe we need to drop it.

## Some text Distribution


In [ ]:
import string

# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))


# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))


### Distribution Graphics

In [ ]:
import seaborn as sns

# Graphic distribution in test & train data for mention count 
sns.set_style("white")
x1 = df_train["mention_count"]
x2 = df_test["mention_count"]


# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})

plt.figure(figsize=(10,7), dpi= 80)
sns.distplot(x1, color="dodgerblue", label="Train", **kwargs)
sns.distplot(x2, color="orange", label="Test", **kwargs)
plt.xlim(0,5)
plt.legend();

In [ ]:
# Graphic distribution in test & train data for hashtag count
sns.set_style("white")
x1 = df_train["hashtag_count"]
x2 = df_test["hashtag_count"]


# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})

plt.figure(figsize=(10,7), dpi= 80)
sns.distplot(x1, color="dodgerblue", label="Train", **kwargs)
sns.distplot(x2, color="orange", label="Test", **kwargs)
plt.xlim(0,5)
plt.legend();

In [ ]:
# Graphic distribution in test & train data for url count
sns.set_style("white")
x1 = df_train["url_count"]
x2 = df_test["url_count"]


# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})

plt.figure(figsize=(10,7), dpi= 80)
sns.distplot(x1, color="dodgerblue", label="Train", **kwargs)
sns.distplot(x2, color="orange", label="Test", **kwargs)
plt.xlim(0,5)
plt.legend();

In [ ]:
# Graphic distribution in test & train data for punctuation count

x1 = df_train["punctuation_count"]
x2 = df_test["punctuation_count"]


# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})

plt.figure(figsize=(10,7), dpi= 80)
sns.distplot(x1, color="dodgerblue", label="Train", **kwargs)
sns.distplot(x2, color="orange", label="Test", **kwargs)
plt.xlim(0,30)
plt.legend();

In [ ]:

x1 = df_train.loc[df_train.target==1,"hashtag_count"]
x2 = df_train.loc[df_train.target==0, "hashtag_count"]


# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})

plt.figure(figsize=(10,7), dpi= 80)
sns.distplot(x1, color="dodgerblue", label="Disaster", **kwargs)
sns.distplot(x2, color="orange", label="No Disaster", **kwargs)
plt.xlim(0,6)
plt.legend();

## Ngrams

### Unigrams


In [ ]:
# Distribution of every words in the train dataset 

from collections import Counter
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import spacy 
nlp=spacy.load("en_core_web_sm")

words=[]
for t in df_train.text:
  for word in t.split():
      words.append(word)
x=(pd.Series(nltk.ngrams(words, 1)).value_counts())[:50]

x.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

The target value can affect the unigrams ?


In [ ]:
df_answer2=df_train[df_train["target"]==0]
words=[]
for t in df_answer2.text:
  for word in t.split():
      words.append(word)
x2=(pd.Series(nltk.ngrams(words, 1)).value_counts())[:50]
x2.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

In [ ]:
#Distribution des mots: 
from collections import Counter
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import spacy 

#Disaster 
nlp=spacy.load("en_core_web_sm")
df_answer=df_train[df_train["target"]==1]
words=[]
for t in df_answer.text:
  for word in t.split():
      words.append(word)
x=(pd.Series(nltk.ngrams(words, 1)).value_counts())[:50]
x.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))


### Bigram

In [ ]:
words=[]
for t in df_train.text:
  for word in t.split():
      words.append(word)
x=(pd.Series(nltk.ngrams(words, 2)).value_counts())[:50]

x.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

### Trigram

In [ ]:
words=[]
for t in df_train.text:
  for word in t.split():
      words.append(word)
x=(pd.Series(nltk.ngrams(words, 3)).value_counts())[:50]

x.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

We need to clean some data as &amp or the MH370 case.
Maybe say something for the distribution of words

## Keywords

In [ ]:
keyw=[]
for i in df_train.keyword:
  keyw.append(i)
counted=Counter(keyw)
x=counted.most_common(150)
#Key
key=[]
for i in range(len(x)):
  key.append(x[i][0])


key

There is a problem with the keywords.The space isn't decoded correctly. We can see in the text some %20

In [ ]:
df_train["keyword"]= df_train["keyword"].astype(str)
for i in range(len(df_train["keyword"])):
  df_train["keyword"].iloc[i]=df_train["keyword"].iloc[i].replace("%20"," ")


df_test["keyword"]= df_test["keyword"].astype(str)
for i in range(len(df_test["keyword"])):
  df_test["keyword"].iloc[i]=df_test["keyword"].iloc[i].replace("%20"," ")



In [ ]:
#Train
keyw=[]
for i in df_train.keyword:
  keyw.append(i)
counted=Counter(keyw)
x=counted.most_common(15)
#Key
key=[]
for i in range(len(x)):
  key.append(x[i][0])

#Values:
values=[]
for i in range(len(x)):
  values.append(x[i][1])
values
data = {'Key': key,
        'Values': values
       }
df = pd.DataFrame(data,columns=['Key','Values'])
df.plot(x ='Key', y='Values', kind = 'bar')
plt.title("Keyword's Distribution")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Test
keyw=[]
for i in df_test.keyword:
  keyw.append(i)
counted=Counter(keyw)
x=counted.most_common(15)
#Key
key=[]
for i in range(len(x)):
  key.append(x[i][0])

#Values:
values=[]
for i in range(len(x)):
  values.append(x[i][1])
values
data = {'Key': key,
        'Values': values
       }
df = pd.DataFrame(data,columns=['Key','Values'])
df.plot(x ='Key', y='Values', kind = 'bar')
plt.title("Keyword's Distribution")
plt.xticks(rotation=70)
plt.show()

We can see Nan in the train set but don't appear on the test set. We are going to check this deeper:

In [ ]:
# Check different distribution of Nan keywords in the different datasets (Test and Train)

Missing_test= df_test.keyword[df_test["keyword"]=="nan"].count()
Missing_train=df_train.keyword[df_train["keyword"]=="nan"].count()
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ["Test", "Train"]
students = [Missing_train,Missing_test]
ax.bar(langs,students)
plt.show()

# Base rate

In [ ]:
# Plot the distribution of Disaster / Not disaster in the training set 
Disaster= df_train.target[df_train["target"]==1].count()
NoDisaster= df_train.target[df_train["target"]==0].count()
#Piechart
labels = 'Disaster', 'Not Disaster'
sizes = [Disaster, NoDisaster]
colors = ['gold', 'lightcoral']
explode = (0.1, 0)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

## Bonus WordCloud

In [ ]:
# WordCloud to visualise occurence of words in the training set where target = 0

no_dis_tweets = df_train[df_train.target == 0]
word = []
for t in no_dis_tweets.text:
    word.append(t)
word = pd.Series(word).str.cat(sep=' ')
from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(word)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# WordCloud to visualise occurence of words in the training set where target = 1

no_dis_tweets = df_train[df_train.target == 1]
word = []
for t in no_dis_tweets.text:
    word.append(t)
word = pd.Series(word).str.cat(sep=' ')
from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(word)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

